In [95]:
def OP(durationExposition=80e-6, durationCoilRamp=500e-6, i=-0.12, pt=3, timeline=None, time=None, **kwargs
      ) :
    if timeline is None :
        return lambda x : OP(durationExposition,durationCoilRamp,i,pt,x,time,**kwargs)

    fullDuration=durationCoilRamp+durationExposition
    if time is None : time=tl.previous_time(timeline)
    print("here!!!",time)
    return tl.stack(
        timeline,
        tl.next(
            coil_MOTlower__A= i,
            coil_MOTupper__A=-i,
            time_start=time,
            t=durationCoilRamp,
            fargs={"ti": pt},
            context="OP",**kwargs
        ),
        tl.set(
            "AOM_OP",[[time+durationCoilRamp,1],[time+fullDuration,0]]
        ),
        tl.set(AOM_repump=0,shutter_repump=0,t=time+fullDuration)
        #     # I switch off the AOM close before opening the first shutter, but safely enough, not to flash too soon
        #     [
        #         "shutter_OP001",
        #         [ time - constants.OP.lag_aom_on - constants.safety_factor * (constants.OP.lag_shutter_on + constants.OP.duration_shutter_on), 1 ],
        #     ],
        #     [
        #         "AOM_OP",
        #         [time - constants.OP.lag_aom_on + 9.5e-6, 1], #TODO: what is this constant?????
        #     ],
        #     [
        #         "shutter_OP002",
        #         [ time + duration_exposure - (2 - constants.safety_factor) * constants.OP.lag_shutter_off, 0 ],
        #     ],
        #     ["AOM_OP", [ time+duration_exposure + 0.5e-6, 0 ] ], # TODO: another mysterious constant
        #     ["AOM_repump", [ time+duration_exposure, 0 ] ],
        #     ["shutter_repump", [ time+duration_exposure, 0 ] ],
        #     # Shutters switched back, to reinitialize them before any additional optical pumpings later.
        #     ["shutter_OP001", [ time+duration_exposure, 0 ] ],
        #     ["shutter_OP002", [ time+duration_exposure + constants.safety_factor * (constants.OP.lag_shutter_on+constants.OP.duration_shutter_on), 1 ] ],
        #     context=context,
        #     timeline=tline,
        ),


In [39]:
exposure=0.002

In [104]:
timeline=tl.stack(
    init(t=-2,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    tl.set(AOM_OP=0),
    molasses(),
#    OP(),
#    expose_camera(15.106,exposure),
#    tl.wait(100e-6),
    magneticTrapping(),
    tl.wait(5e-3),
    pull_coils(1e-3,0.,0.),
    imaging_absorption(15.117,exposure,exposureBlow=1,delayBg=200e-3,delayLi=3),
    tl.wait(1),
    tl.next(
        lockbox_MOT__MHz=0,
        coil_MOTlower__A=-1.0,
        coil_MOTupper__A=-0.98,
        t=.1,
        fargs={"ti": 3},context='sanitize'
    ),
    tl.set(AOM_MOT=1,AOM_repump=1,AOM_imaging=1)
)#)

In [105]:
tl.display(timeline)

In [111]:
timeline=tl.stack(
    init(t=-2,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
    MOT(t=0.0),
)

In [112]:
output=adwin.output(adwin.add(timeline,connections,devices))

In [113]:
m = ADwin.ADwin(1)
m = adwin.initialize_ADwin(m, output)

time_end: 0.0s


In [108]:
image=take_image_absorption_camera(m, processNo=1,nmb_ims=4,CameraExposureTime_s=exposure,print_log=True)

INITIAL VALUES
fps: 0.0
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 84
Exposition time: 19.25361403508772 ms
AOI: <pypyueye.utils.Rect object at 0x000001B9E9E82350>

MODIFIED VALUES
fps: 59.822673002972245
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 86
Exposition time: 1.9998596491228071 ms

AFTER CAPTURE VALUES
Framerate: 59.822673002972245
Frame range: 0.9472267726036225, 59.82267300297225
AOI:1024,1280


In [109]:
absIm=np.log((ab.replaceNaNwith0(image[2])-ab.replaceNaNwith0(image[3]))/(ab.replaceNaNwith0(image[0])-ab.replaceNaNwith0(image[1])))[0:600,670:1200]

C:\Users\User\AppData\Local\Temp\ipykernel_3356\4227516746.py:1: RuntimeWarning: divide by zero encountered in divide
  absIm=np.log((ab.replaceNaNwith0(image[2])-ab.replaceNaNwith0(image[3]))/(ab.replaceNaNwith0(image[0])-ab.replaceNaNwith0(image[1])))[0:600,670:1200]
C:\Users\User\AppData\Local\Temp\ipykernel_3356\4227516746.py:1: RuntimeWarning: invalid value encountered in divide
  absIm=np.log((ab.replaceNaNwith0(image[2])-ab.replaceNaNwith0(image[3]))/(ab.replaceNaNwith0(image[0])-ab.replaceNaNwith0(image[1])))[0:600,670:1200]
C:\Users\User\AppData\Local\Temp\ipykernel_3356\4227516746.py:1: RuntimeWarning: divide by zero encountered in log
  absIm=np.log((ab.replaceNaNwith0(image[2])-ab.replaceNaNwith0(image[3]))/(ab.replaceNaNwith0(image[0])-ab.replaceNaNwith0(image[1])))[0:600,670:1200]
C:\Users\User\AppData\Local\Temp\ipykernel_3356\4227516746.py:1: RuntimeWarning: invalid value encountered in log
  absIm=np.log((ab.replaceNaNwith0(image[2])-ab.replaceNaNwith0(image[3]))/(ab.r

In [110]:
plt.figure()
plt.imshow(absIm, interpolation='None',cmap="viridis")
plt.colorbar()
plt.grid(False)

In [291]:
delay = np.arange(0, 11e-3, 1e-3)
timeline = []

timeline_prepare = tl.stack(
    init(t=-2,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    molasses(),
    tl.set(AOM_repump=0,shutter_repump=0,AOM_imaging=0)
)#)

for d in delay:
    timeline.append(
        tl.stack(
            timeline_prepare,
            tl.set(AOM_repump=0,shutter_repump=0,AOM_imaging=0),
            #    expose_camera(15.106,exposure),
            imaging_absorption(15.106+d,exposure,exposureBlow=1,delayBg=200e-3,delayLi=3),
            tl.wait(1),
            tl.next(
                lockbox_MOT__MHz=0,
                coil_MOTlower__A=-1.0,
                coil_MOTupper__A=-0.98,
                t=.1,
                fargs={"ti": 3},context='sanitize'
            ),
            tl.set(AOM_MOT=1,AOM_repump=1,AOM_imaging=1)
        )
    )

In [292]:
tl.display(timeline[10])

In [296]:
image=[]
for i in range (len(delay)):
    output=adwin.output(adwin.add(timeline[i],connections,devices))
    m = ADwin.ADwin(1)
    m = adwin.initialize_ADwin(m, output)
    image.append(take_image_absorption_camera(m, processNo=1,nmb_ims=4,CameraExposureTime_s=exposure,print_log=True))

time_end: 19.4082s
INITIAL VALUES
fps: 0.0
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 84
Exposition time: 19.25361403508772 ms
AOI: <pypyueye.utils.Rect object at 0x00000252058640D0>

MODIFIED VALUES
fps: 59.822673002972245
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 86
Exposition time: 1.9998596491228071 ms

AFTER CAPTURE VALUES
Framerate: 59.822673002972245
Frame range: 0.9472267726036225, 59.82267300297225
AOI:1024,1280
time_end: 19.409200000000002s
INITIAL VALUES
fps: 0.0
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 84
Exposition time: 19.25361403508772 ms
AOI: <pypyueye.utils.Rect object at 0x0000025205C104D0>

MODIFIED VALUES
fps: 59.822673002972245
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 86
Exposition time: 1.9998596491228071 ms

AFTER CAPTURE VALUES
Framerate: 59.822673002972245
Frame range: 0.9472267726036225, 59.82267300297225
AOI:1024,1280
time_end: 19.4102

In [311]:
absim=[]

for i in range(len(delay)):
    absim.append(np.log((ab.replaceNaNwith0(image[i][2][0:600,670:1200])-ab.replaceNaNwith0(image[i][3][0:600,670:1200]))/(ab.replaceNaNwith0(image[i][0][0:600,670:1200])-ab.replaceNaNwith0(image[i][1][0:600,670:1200]))))

In [319]:
len(absim)

11

In [320]:
np.savez("D://Data/2024-10-15_WignerTimeTest_WithAbsorptionImaging/ExpansionAfterMolassesWith1msStep",*absim)

In [321]:
len(np.load("D://Data/2024-10-15_WignerTimeTest_WithAbsorptionImaging/ExpansionAfterMolassesWith1msStep.npz"))

11

In [312]:
plt.figure()

for i in range (len(delay)):
    plt.subplot(3, 5, i+1)
    plt.imshow(absim[i], interpolation='None',cmap="viridis")
    plt.colorbar()
    plt.grid(False)

In [317]:
plt.figure()

for i in range (len(delay)):
    plt.plot(np.sum(absim[i],axis=1), label=f'{i}')

plt.legend(title='Delay [ms]')

In [310]:
np.sum(absim[0],axis=0)

-6.369883219300329

In [243]:
#tl.display(
timeline=tl.stack(
    init(t=-2,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
    MOT(t=0.0),
    MOT_detunedGrowth(),
    molasses(),
    tl.set(AOM_repump=0,shutter_repump=0,AOM_imaging=0),
#    expose_camera(15.106,exposure),
    imaging_absorption(15.106,exposure,exposureBlow=1,delayBg=200e-3,delayLi=3),
    tl.wait(1),
    tl.next(
        lockbox_MOT__MHz=0,
        coil_MOTlower__A=-1.0,
        coil_MOTupper__A=-0.98,
        t=.1,
        fargs={"ti": 3},context='sanitize'
    ),
    tl.set(AOM_MOT=1,AOM_repump=1,AOM_imaging=1)
)#)

In [236]:
timeline=tl.stack(
    init(t=-2,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
    trigger_camera(1,exposure,context="AI"),
    trigger_camera(1.02,exposure,context="AI"),
    tl.wait(1,context="AI")
)

In [244]:
tl.display(timeline)

In [101]:
output

[[(-2, 3, 8, 32768),
  (-2, 4, 7, 35499),
  (-2, 3, 2, 49152),
  (-2, 4, 2, 33423),
  (-2, 4, 4, 32113),
  (2200400, 4, 4, 32113),
  (2200400, 4, 2, 33423),
  (2200400, 3, 2, 49152),
  (2200400, 4, 7, 35499),
  (2200400, 3, 8, 32768)],
 [(-2, 31, 1),
  (-2, 2, 1),
  (-2, 1, 1),
  (-2, 12, 0),
  (-2, 14, 0),
  (-2, 15, 1),
  (-2, 13, 0),
  (-2, 5, 1),
  (-2, 0, 0),
  (-2, 11, 0),
  (999999, 0, 1),
  (1000399, 0, 0),
  (1999999, 0, 1),
  (2000400, 0, 0),
  (2200400, 5, 1),
  (2200400, 13, 0),
  (2200400, 15, 1),
  (2200400, 14, 0),
  (2200400, 12, 0),
  (2200400, 1, 1),
  (2200400, 2, 1),
  (2200400, 31, 1),
  (2200400, 0, 0),
  (2200400, 11, 0)]]

In [247]:
image=[]

In [248]:
image=take_image_absorption_camera(m, processNo=1,nmb_ims=4,CameraExposureTime_s=exposure,print_log=True)

INITIAL VALUES
fps: 0.0
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 84
Exposition time: 19.25361403508772 ms
AOI: <pypyueye.utils.Rect object at 0x000002526FAB5350>

MODIFIED VALUES
fps: 59.822673002972245
Available fps range: [0.9472267726036225, 59.82267300297225]
Pixelclock: 86
Exposition time: 1.9998596491228071 ms

AFTER CAPTURE VALUES
Framerate: 59.822673002972245
Frame range: 0.9472267726036225, 59.82267300297225
AOI:1024,1280


In [249]:
len(image)

4

In [135]:
#crop_image = ab.replaceNaNwith0(image[0][200:800,750:950])
crop_image = ab.replaceNaNwith0(image[0][233:533,854:1054])
full_image = ab.replaceNaNwith0(image[0])

In [137]:
full_image = ab.replaceNaNwith0(image[1])

In [304]:
fullimages=[]
for i in range(4):
    fullimages.append(ab.replaceNaNwith0(image[0][i][0:600,670:1200]))

In [305]:
plt.figure()

plt.subplot(2, 2, 1)
plt.imshow(fullimages[0], interpolation='None',cmap="viridis")
plt.title('At')
plt.colorbar()
plt.grid(False)

plt.subplot(2, 2, 2)
plt.imshow(fullimages[1], interpolation='None',cmap="viridis")
plt.title('Bg_At')
plt.colorbar()
plt.grid(False)

plt.subplot(2, 2, 3)
plt.imshow(fullimages[2], interpolation='None',cmap="viridis")
plt.title('Li')
plt.colorbar()
plt.grid(False)

plt.subplot(2, 2, 4)
plt.imshow(fullimages[3], interpolation='None',cmap="viridis")
plt.title('Bg_Li')
plt.colorbar()
plt.grid(False)

#plt.imshow(crop_image, interpolation='None',cmap="viridis")
#plt.plot(100,150,'ro')
#plt.imshow(full_image, interpolation='None',cmap="viridis")

#plt.plot(954,383,'ro')
#plt.plot(867,277,'ro')

In [306]:
absim = np.log((fullimages[2]-fullimages[3])/(fullimages[0]-fullimages[1]))

In [307]:
plt.figure()
plt.imshow(absim, interpolation='None',cmap="viridis")
plt.colorbar()
plt.grid(False)

# Imports, constants, sample-preparation functions

In [1]:
import sys
import time
#sys.path.append("..")

import importlib.resources as resources
import pathlib as pl
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from munch import Munch
from wigner_time import connection as con
from wigner_time import timeline as tl
from wigner_time import adwin as adwin
from wigner_time import constructor as construct
import wigner_time
from wigner_time import absorptionimaging as ab

from copy import deepcopy

import ADwin

from pyueye import ueye
from pypyueye import Camera

%matplotlib qt

In [2]:
connections = con.connection(
    ["shutter_MOT", 1, 11],
    ["shutter_repump", 1, 12],
    ["shutter_OP001", 1, 14],
    ["shutter_OP002", 1, 15],
    ["AOM_MOT", 1, 1],
    ["AOM_repump", 1, 2],
#    ["AOM_OP_aux", 1, 30],  # should be set to 0 always
    ["AOM_OP", 1, 31],
    ["coil_compensationX__A", 4, 7],
    ["coil_compensationY__A", 3, 2],
    ["coil_MOTlower__A", 4, 1],
    ["coil_MOTupper__A", 4, 3],
    ["coil_MOTlowerPlus__A", 4, 2],
    ["coil_MOTupperPlus__A", 4, 4],
    ["lockbox_MOT__MHz", 3, 8],
)

devices = pd.DataFrame(
    columns=["variable", "unit_range", "safety_range"],
    data=[
        ["coil_compensationX__A", (-3, 3), (-3, 3)],
        ["coil_compensationY__A", (-3, 3), (-3, 3)],
        ["coil_MOTlower__A", (-5, 5), (-5, 5)],
        ["coil_MOTupper__A", (-5, 5), (-5, 5)],
        ["coil_MOTlowerPlus__A", (-5, 5), (-5, 5)],
        ["coil_MOTupperPlus__A", (-5, 5), (-5, 5)],
        #["lockbox_MOT__V", (-10, 10)],
        ["lockbox_MOT__MHz", (-200, 200)],
    ],
)

# I dislike global variables but, unfortunately, a reference will probably still need to be available in the same namespace as these functions for convenience.
constants = Munch(
    safety_factor=1.1,
#    factor__VpMHz=0.05,
    lag_MOTshutter=2.48e-3,
    Compensation=Munch(
        Z__A=-0.1,
        Y__A=1.5,
        X__A=0.25,
    ),
    OP=Munch(
        lag_AOM_on=15e-6,
        lag_shutter_on=1.5e-3,
        lag_shutter_off=1.5e-3,
        duration_shutter_on=140e-6,
        duration_shutter_off=600e-6,
    ),
    AI=Munch(
        lag_shutter_on=2.2e-3,
        lag_shutter_off=1.9e-3,
    )
)


In [68]:
def init(**kwargs) :
    """
    Creates an experimental timeline for the initialization of every device.
    """
    return tl.create(
        lockbox_MOT__MHz=0.0,
        coil_compensationX__A= constants.Compensation.X__A,
        coil_compensationY__A= constants.Compensation.Y__A,
        coil_MOTlowerPlus__A = -constants.Compensation.Z__A,
        coil_MOTupperPlus__A =  constants.Compensation.Z__A,
        AOM_MOT = 1,
        AOM_repump = 1,
#        AOM_OP_aux = 0, # TODO: USB-controlled AOMs should be treated on a higher level
        AOM_OP = 1,
        shutter_MOT = 0,
        shutter_repump = 0,
        shutter_OP001 = 0,
        shutter_OP002 = 1,
        context="ADwin_LowInit",
        **kwargs
    )


def MOT(duration=15, lA=-1., uA=-0.98, **kwargs) :
    return tl.stack(
        tl.set(
#           waitChannel = 0,
            shutter_MOT=1,
            shutter_repump=1,
            coil_MOTlower__A=lA,
            coil_MOTupper__A=uA,
            context='MOT',**kwargs
        ),
        tl.wait(duration)
    )
    
def MOT_detunedGrowth(duration=100e-3, durationRamp=10e-3, toMHz=-5, pt=3, **kwargs) :
    return tl.stack(
        tl.next(
            lockbox_MOT__MHz=toMHz,
            t=durationRamp,
            fargs={"ti": pt},context='MOT',**kwargs
        ),
        tl.wait(duration-durationRamp,#"lockbox_MOT__MHz",
                )
    )

def molasses(duration=5e-3, durationCoilRamp=9e-4, durationLockboxRamp=1e-3, toMHz=-90, coil_pt=3, lockbox_pt=3, **kwargs) :

    return tl.stack(
        tl.next(
            coil_MOTlower__A=0,
            coil_MOTupper__A=0, # TODO: can these be other than 0 (e.g. for more perfect compensaton?)
            t=durationCoilRamp,
            fargs={"ti": coil_pt},
            context="molasses",
            **kwargs
        ),
        tl.next(
            lockbox_MOT__MHz=toMHz,
            t=durationLockboxRamp,
            fargs={"ti": lockbox_pt}
        ),
        tl.set(
            shutter_MOT=[duration - constants.lag_MOTshutter,0],
            AOM_MOT=[duration,0],
            relative=True),
        tl.wait(1e-5)
    )

def pull_coils(duration, l, u, lp=-constants.Compensation.Z__A, up=constants.Compensation.Z__A, pt = 3, **kwargs) :
    return tl.next(
        coil_MOTlower__A=l,
        coil_MOTupper__A=u,
        coil_MOTlowerPlus__A=lp,
        coil_MOTupperPlus__A=up,
        fargs={"ti": pt},
        t=duration,
        **kwargs,
    )

def magneticTrapping(durationInitial=500e-6, li=-1.8, ui=-1.7, durationStrengthen=3e-3, ls=-4.8, us=-4.7, **kwargs) :
    return tl.stack(
        pull_coils(durationInitial,li,ui,**kwargs),
        pull_coils(durationStrengthen,ls,us)
    )

## Absorption imaging

In [4]:
connections=pd.concat([connections,con.connection(["shutter_imaging", 1, 13],["AOM_imaging", 1, 5],["trigger_camera", 1, 0])])

In [5]:
def trigger_camera(time, exposure, **kwargs) :
    return tl.set(
        ["trigger_camera",[time,1]],
        ["trigger_camera",[time+exposure,0]],
        **kwargs)

def flash_light(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        tl.set(
            ["AOM_imaging",[time,1]],
            ["AOM_imaging",[time+exposure,0]],
            **kwargs),
        tl.set(
            ["shutter_imaging",[time-exposure*(sf-1)-constants.AI.lag_shutter_on,1]],
            ["shutter_imaging",[time+exposure*sf,0]]
        ), # TODO: what to write here exactly?
    )

def expose_camera(time, exposure, **kwargs) :
    sf = constants.safety_factor
    return tl.stack(
        flash_light(time,exposure,**kwargs),
        trigger_camera(time-exposure*(sf-1)/2, exposure*sf)
    )

def take_image_plus_Bg(time, exposure, delayBg, **kwargs) :
    return tl.stack(
        expose_camera(time, exposure, **kwargs), # taking At image
        trigger_camera(time+delayBg, exposure*constants.safety_factor), # taking Bg_At image
    )    

def imaging_absorption(time, exposure, delayBg, delayLi, exposureBlow=None, **kwargs) :
    """
    Creates an experimental timeline for absorption imaging.
    """
    context = "imaging_absorption"
    return tl.stack(
        tl.set(AOM_imaging=[time-constants.safety_factor*exposure,0],context=context,**kwargs), # initializing the AOM
        take_image_plus_Bg(time, exposure, delayBg), # taking At + Bg_At image
        flash_light(time+5*delayBg,exposureBlow) if exposureBlow is not None else None, # blow out the atoms in between
        take_image_plus_Bg(time+delayLi, exposure, delayBg), # taking Li + Bg_Li image
    )

# High-level dirver (camera, ADwin, etc.)

In [6]:
def take_image_absorption_camera(m, processNo,nmb_ims=1,CameraExposureTime_s=0.0002,aoi=(0,0,1280,1024), print_log  = False) :

    with Camera(device_id=1, buffer_count=nmb_ims) as cam :
        cam.set_colormode(ueye.IS_CM_MONO8)
        #
        if print_log:
            print(f"INITIAL VALUES")
            print(f'fps: {cam.get_fps()}')
            print(f'Available fps range: {cam.get_fps_range()}')
            print(f'Pixelclock: {cam.get_pixelclock()}')
            print("Exposition time:",cam.get_exposure(),'ms')
        cam.set_aoi(*aoi)
        
        cam.set_pixelclock(86)
        cam.set_fps(60)
        cam.set_exposure(CameraExposureTime_s*1000) #exposuretime in ms
        if print_log:
            print("AOI:",cam.get_aoi())
            print("")
            print(f"MODIFIED VALUES")
            print(f'fps: {cam.get_fps()}')
            print(f'Available fps range: {cam.get_fps_range()}')
            print(f'Pixelclock: {cam.get_pixelclock()}')
            print("Exposition time:",cam.get_exposure(),'ms')

       # #==============================================================================
       # # Capturing images in memory
       # #==============================================================================
        a = time.time()
        ueye.is_SetExternalTrigger(cam.h_cam, ueye.IS_SET_TRIGGER_LO_HI) #here I set the external trigger

        m.Start_Process(processNo)
        ret = cam.capture_images(nmb_ims,60000)                  #this does the image capturing
        
        fps = cam.get_fps()
        fpsr = cam.get_fps_range()
        if print_log:
            print("")
            print(f"AFTER CAPTURE VALUES")
            print(f"Framerate: {fps}")
            #print(f'Measured framerate: {nmb_ims/(time.time() - a)}') #doesn't work with the new system because of the Li pics
            print(f"Frame range: {fpsr[0]}, {fpsr[1]}")
            print(f"AOI:{cam.get_aoi().height},{cam.get_aoi().width}")
        
    while m.Process_Status(processNo)==1 : time.sleep(0.1)
            
    return [x.astype(int) for x in ret]

# Old stuff, test, etc.

In [61]:
timeline = tl.stack(
    init(t=-0.2,shutter_imaging=0,AOM_imaging=1,trigger_camera=0),
    expose_camera(time=1,exposure=exposure,context="AI")
#    tl.wait(10)
)

In [125]:
tl.display(timeline)

In [149]:
plt.figure()
#sumx=np.sum(crop_image,axis=1)
sumx=np.sum(full_image,axis=1)
plt.vlines(383,min(sumx),max(sumx),color='r')
plt.plot(sumx);

In [150]:
plt.figure()
#sumy=np.sum(crop_image,axis=0)
sumy=np.sum(full_image,axis=0)
plt.vlines(954,min(sumy),max(sumy),color='r')
plt.plot(sumy);